### AppDev Part two: Laoyout Design
by: Alireza Dibazar
dibazar@gmail.com

# Summary:

We will learn to develope an app for live streaming of IoT data. The app is suppose to provide analytics-based anomaly detection with details needed to guide field engineers. The data is fetched from Predix Timeseries database and it provides readings of a Three-axis accelerometer which installed in a gas turbine. It is suggested to see the deployed version of the app before the exercise. The app can be found at the following url: https://sample-live-streaming.run.aws-usw02-pr.ice.predix.io/ and [document here](./Documents/Sample_live_Stream.pptx) (Sample_live_Stream.pptx)

### Objectives:
1. Learn planning of the project based of agile methodology
2. Learn how to read data from Cassandra -- Predix Timeseries database -- and be able to change sampling frequency of the data
3. Be able to visually demonstrate field engineers pain point and convert that to a wireframe
4. Design analytics
5. Learn to design app layout based on wireframe
6. Be able to deply the app and operationalize it for users


### This Notebook
This notebook focuses on designing a sample layout. It consists of multiple boxes to host charts, radio buttons, and texts. Layout of app should be prepared after having full understanding of user(s) needs and experinces. For this project, students can consider designing their own lauouts with supporting rationals.

We are using Plotly's "Dash" for the design. Please consider visiting https://dash.plot.ly/, https://dash.plot.ly/getting-started, and reading through Dash's "User Guide" as a pre-requisit. After reading the material please have basic understanding of 1) layout, 2) callbacks, and 3) Dash core components.

Please note that in this notebook we wont be working on callbacks. The callbacks will be addressed in Part 3 when all components of the App are assembled together.

### Assignment
1. Change font color and size
2. Change window and background colors
3. Create a drop-down list and move content of dcc.RadioItems with of id='historical-data' into that
4. Reassign proper names to Id(s); the employed names in this example layout may not represent the functions


In [137]:
# This section imports necessary packages 
# The app employs two major components: "flask" and plotly's "dash"

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import chart_studio.plotly as py
from plotly.graph_objs import *
from flask import Flask
import os

In [138]:
import os

In [139]:
external_css = ["https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "https://fonts.googleapis.com/css?family=Raleway:400,400i,700,700i",
                "https://fonts.googleapis.com/css?family=Product+Sans:400,400i,700,700i",
               "temlate.css"]

app = dash.Dash(
    'streaming-data-app',
    external_stylesheets=external_css
)

server = app.server

app.css.config.serve_locally = True
app.scripts.config.serve_locally = True

In [140]:
# layout is defined here
# 

app.layout = html.Div([
    html.Div([
    
        html.H2("Live Turbine Vibration Monitoring"),
    ], className='banner'),
            html.Div([
                html.Div([
                    dcc.RadioItems(
                        id='select-mode',
                        options=[{'label': i, 'value': i} for i in ['Threshold', 'Analytic 1','Analytic 2']],
                        value='Threshold',
                        labelStyle={'display': 'inline-block'}
                    )],
                    className='five columns'),
                html.Div([
                    dcc.RadioItems(id='Start-Stop-stream', 
                    options= [{'label':'Start-Stream', 'value':'STR'},
                                {'label':'Stop-Stream', 'value':'STP'}],
                                   value='STR',
                           labelStyle={'display': 'inline-block'})
                        ], style={'color':'aliceblue'},
                            className='five columns offset-by-five'),
                
                html.Div([
                    dcc.Dropdown(id='data-stream-type',
                                options = [
                                            {'label':'Historical-Data','value':'2018'},
                                            {'label':'Live-Stream','value':'live'}
                                          ]
                                )], style = {'color':'rgb(35, 38, 36)'},
                                    className='five columns offset-by-four'),
                    
                    
                html.Div([html.H4(" ")],className='two columns',id='live-update-text')
                
            ], className='row'),
    
    html.Div([
        html.Div([
            html.H5(" ")
        ], className='Title'),
        
        html.Div([
            dcc.Graph(id='main-chart'),
        ], className='twelve columns layout'),
#         
#         Read data every 60 secs and refresh the charts accordingly
#         
        dcc.Interval(id='speed-update', interval=60000, n_intervals=0), 
    ], className='row speed-row', style={'backgroundColor': 'rgb(112, 112, 112)'}),

    html.Div([
        html.H5("  Accelartion Magnitude  ")
        ], className='Row Title'),
    html.Div([
            dcc.Graph(id="Reactive-Power00"),
        ], className='twelve columns speed'),
#########
    html.Div([
    html.Div([
            html.Div([
                html.H5("Acc1 data(Top) and Acc2 data (Bottom)")
            ], className='Title'),
            dcc.Graph(id='Turb_Brg_Oil_temp'),
            dcc.Graph(id='Jet-Pumps'),
            ], className='six columns'),
        
        html.Div([
            html.Div([
                html.H5("Acc3 data(Top) and Acc4 data (Bottom)")
            ], className='Title'),
            dcc.Graph(id='TG_Brg_Temp'),
            dcc.Graph(id='TG_Brg_Vib'),
        ], className='six columns')
    ], className='row'),   

    html.Div([
        html.H2(" -- ")
        ], className='Row Title'),
        
    
], style={'padding': '0px 10px 15px 10px',
          'backgroundColor': 'rgb(35, 38, 36)',
          'marginLeft': 'auto', 'marginRight': 'auto', "width": "1600px",
          'boxShadow': '0px 0px 5px 5px rgba(204,204,204,0.4)',
             'font-size': '20px',
             'color': 'aliceblue'})


In [141]:
services = os.getenv("VCAP_SERVICES")
port = None

if services is not None:
    vcap = json.loads(services)
    port = int(os.getenv("PORT"))

In [142]:
if __name__ == '__main__':
    if port is not None:
        app.run_server(host='0.0.0.0', port=port)

    else:
        app.run_server()


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.0&m=1568981304 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET /temlate.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.0&m=1568981304 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.0&m=1568981304 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:55] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1568981306 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:56] "GET /_dash-component-suites/dash_core_components/plotly-1.49.4.min.js?v=1.2.1&m=1568981305 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2019 01:07:56] "GET /_dash-component-suites/dash_core_components/highl